In [19]:
#Duval algorithm 
#outputs lyndon words of same depth, i.e. # of y, when you are using y_i words

n = 5 #length of the output words
S = ['x', 'y']
k = len(S) 
S.sort() 
  
# To store the indices 
# of the characters 
w = [-1]
  
# Loop till w is empty 
while w: 
  
    # Incrementing the last character 
    w[-1] += 1
    m = len(w) 
    if m == n: 
        print(''.join(S[i] for i in w)) 
        print(w)
    
    # Repeating w to get a 
    # n-length string 
    while len(w) < n: 
        w.append(w[-m]) 
    
    # Removing the last character 
    # as long it is equal to 
    # the largest character in S 
    while w and w[-1] == k - 1: 
        w.pop() 

xxxxy
[0, 0, 0, 0, 1]
xxxyy
[0, 0, 0, 1, 1]
xxyxy
[0, 0, 1, 0, 1]
xxyyy
[0, 0, 1, 1, 1]
xyxyy
[0, 1, 0, 1, 1]
xyyyy
[0, 1, 1, 1, 1]


In [2]:
def plus1(u, v): #v will CHANGE to u+v; u won't change
    for k in u:
        if k in v:
            v[k] += u[k]
        else:
            v[k] = u[k]
            
def scal1(n, d): #d will CHANGE    
    for k in d:
        d[k] *= n

def concat(u, v): #u,v are dicts with xy-string/z-value-tuple keys; return uv
                    #watch out when u, v are empty dicts   
    r = {}
    for k in u:
        for l in v:           
            r[k + l] = u[k] * v[l] #k + l joins two strings or tuples
    return r

def l(u, v):
    r = concat(v,u)
    scal1(-1, r)
    plus1(concat(u,v), r)
    return r

class z:
    
    #PUT THE LAST PART INTO THE if m == ny PART
    #generate lyndon words of depth ny
    #the alphabet set = {1,2,3,...,nx=wt-dp}, order is 1 < 2 < 3 < ...
    def lyndon_words(nx, ny):

        #this part is added after the code check
        if nx == 0:
            if ny == 1:
                return [(1,)]
            else:
                return []
        if ny == 0:
            return []

        output = []
        w = [0]

        # Loop till w is empty 
        while w: 

            # Incrementing the last character 
            w[-1] += 1
            m = len(w) 
            if m == ny:
                #w is lyndon. Output it
                output.append(tuple(w)) #if w is not changed to tuple, will cause problems

            # Repeating w to get an ny-length string 
            while len(w) < ny: 
                w.append(w[-m]) 

            # Removing the last character 
            # as long it is equal to 
            # the largest letter in the alphabet set
            while w and w[-1] == nx: 
                w.pop()

        a = []
        for i in range(ny-1):
            a.append(1)
        a.append(nx+1)
        trimmed_output = [tuple(a)]

        #discard words not having weight wt0 = nx + ny
        wt0 = nx + ny    
        for i in range(len(output)):
            wt = 0
            for j in output[i]:
                wt += j

            if wt == wt0:
                trimmed_output.append(output[i])

        return trimmed_output #still follows the lexicographic order
### end of class z ###

def is_lyndon(t): #t: tuple; returns True if t is a letter
    for i in range(1, len(t)): #the last i is len(t)-1
        if t >= t[i:]: return False
    return True

def standard_fac(t): #t: lyndon tuple; returns nothing if t is a letter
    for i in range(1, len(t)):
        if is_lyndon(t[:i]) and is_lyndon(t[i:]):      
            return (t[:i], t[i:])
        
def lyn_lambda(t): #t: lyndon tuple
    if len(t) == 1:
        return t[0]
    
    t0, t1 = standard_fac(t)
    return [lyn_lambda(t0), lyn_lambda(t1)]

def lyn_lie_poly(t): #t: lyndon tuple
    if len(t) == 1:
        return {t: 1}
    
    t0, t1 = standard_fac(t)
    return l( lyn_lie_poly(t0) , lyn_lie_poly(t1) )

class xy:
    def lyndon_words(nx, ny):
        if nx == ny == 0: return []
        
        wt = nx + ny
        output = []
        w = [-1]

        # Loop till w is empty 
        while w: 

            # Incrementing the last character 
            w[-1] += 1
            m = len(w) 
            if m == wt:                
                #get the number of 1's in w
                ny_in_w = 0
                for i in w: ny_in_w += i
                
                if ny_in_w == ny:
                    output.append(tuple(w))

            # Repeating w to get a wt-length string 
            while len(w) < wt: 
                w.append(w[-m]) 

            # Removing the last character 
            # as long it is equal to 
            # the largest letter in the alphabet set
            while w and w[-1] == 1: 
                w.pop()
                
        return output
    
    def toz(d): #this is the projection map pi_y. 程式碼參考自mzv.py中的toz
        # e.g. k = (0,0,1,0,1,1,0,0,0,1). place of 1's: 2, 4, 5, 9. The result is (3,2,1,4) = (2+1,4-2,5-4,9-5). 

        r = {}
        for k in d:
            if k[-1] == 0: continue
            
            l = []
            #l records position of 1's
            for i in range(len(k)): #i=0 ~ len(k)-1
                if k[i] == 1:
                    l.append(i)

            ll = [ l[0] + 1 ]
            for i in range(0, len(l)-1): # 0 =< i =< (len(l)-1)-1 = len(l)-2
                ll.append(l[i+1] - l[i])

            t = tuple(ll)
            r[t] = d[k]
            
        return r

### end of class xy ###

def compare(nx, ny):

    lxy = xy.lyndon_words(nx, ny)
    lz = z.lyndon_words(nx, ny)
    llxy_before_toz = [ lyn_lie_poly( t ) for t in lxy ] #basis of the primitive elements of xy-shuffle (with nx x's and ny y's)
    llxy = [ xy.toz(lyn_lie_poly( t )) for t in lxy ] 
    llz =  [ lyn_lie_poly( t ) for t in lz ] #basis of the primitive elements of z-value-shuffle (with nx x's and ny y's)
    
    print('primitive elements of xy-shuffle:')
    print(llxy_before_toz)
    print('pi_y( primitive elements of xy-shuffle ):')
    print(llxy)
    print('primitive elements of z-value-shuffle:')
    print(llz)

In [4]:
z.lyndon_words(6,2)

[(1, 7), (2, 6), (3, 5)]

In [42]:
print( type(lyndon_words(5,3)[1][0:2]) )
print( (0,1) < (0,) )
print( (0,1,3,2)[3:] )

<class 'tuple'>
False
(2,)


In [64]:
print( (1,1)[1:] )
print( (1,1) < (1,) )

(1,)
False


In [17]:
is_lyndon((3, 4, 4))

True

In [93]:
x = {
    'x': 1
}
y = {
    'y': 1
}
l(x,y)

{'yx': -1, 'xy': 1}

In [10]:
lyn_lambda((0,0,1,1))

[0, [[0, 1], 1]]

In [39]:
xy.toz(lyn_lie_poly( (0,0,1,1) ))

{(2, 2): -2, (3, 1): 1}

In [60]:
compare(1,2)

primitive elements of xy-shuffle:
[{(1, 1, 0): 1, (1, 0, 1): -2, (0, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 2): -2, (2, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1): -1, (1, 2): 1}]


In [62]:
compare(0,3)

primitive elements of xy-shuffle:
[]
pi_y( primitive elements of xy-shuffle ):
[]
primitive elements of z-value-shuffle:
[]


In [63]:
compare(2,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0): -1, (1, 0, 1, 0): 2, (0, 1, 1, 0): 0, (0, 1, 0, 1): -2, (0, 0, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(2, 2): -2, (3, 1): 1}]
primitive elements of z-value-shuffle:
[{(3, 1): -1, (1, 3): 1}]


In [57]:
compare(1,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0): -1, (1, 1, 0, 1): 3, (1, 0, 1, 1): -3, (0, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 1, 2): 3, (1, 2, 1): -3, (2, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1, 1): 1, (1, 2, 1): -2, (1, 1, 2): 1}]


In [58]:
compare(3,1) #lambda(xxxy) satisfies the defining properties of ls, but is purposedly excluded from ls.
#so this element does not contradict the parity result (Corollary 3.4.4) even if nx is odd.

primitive elements of xy-shuffle:
[{(1, 0, 0, 0): -1, (0, 1, 0, 0): 3, (0, 0, 1, 0): -3, (0, 0, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(4,): 1}]
primitive elements of z-value-shuffle:
[{(4,): 1}]


In [59]:
compare(4,1) #as above

primitive elements of xy-shuffle:
[{(1, 0, 0, 0, 0): 1, (0, 1, 0, 0, 0): -4, (0, 0, 1, 0, 0): 6, (0, 0, 0, 1, 0): -4, (0, 0, 0, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(5,): 1}]
primitive elements of z-value-shuffle:
[{(5,): 1}]


In [3]:
compare(3,2)

primitive elements of xy-shuffle:
[{(1, 1, 0, 0, 0): 1, (1, 0, 1, 0, 0): -2, (0, 1, 1, 0, 0): -1, (0, 1, 0, 1, 0): 4, (0, 0, 1, 1, 0): -1, (0, 0, 1, 0, 1): -2, (0, 0, 0, 1, 1): 1}, {(1, 0, 1, 0, 0): 1, (1, 0, 0, 1, 0): -3, (1, 0, 0, 0, 1): 2, (0, 1, 1, 0, 0): -1, (0, 1, 0, 1, 0): 4, (0, 1, 0, 0, 1): -3, (0, 0, 1, 1, 0): -1, (0, 0, 1, 0, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(3, 2): -2, (4, 1): 1}, {(1, 4): 2, (2, 3): -3, (3, 2): 1}]
primitive elements of z-value-shuffle:
[{(4, 1): -1, (1, 4): 1}, {(3, 2): -1, (2, 3): 1}]


In [4]:
compare(2,3)

primitive elements of xy-shuffle:
[{(1, 1, 1, 0, 0): 1, (1, 1, 0, 1, 0): -3, (1, 0, 1, 1, 0): 3, (0, 1, 1, 1, 0): -2, (0, 1, 1, 0, 1): 3, (0, 1, 0, 1, 1): -3, (0, 0, 1, 1, 1): 1}, {(1, 1, 0, 1, 0): 1, (1, 1, 0, 0, 1): -1, (1, 0, 1, 1, 0): -3, (1, 0, 1, 0, 1): 4, (0, 1, 1, 1, 0): 2, (0, 1, 1, 0, 1): -3, (1, 0, 0, 1, 1): -1, (0, 1, 0, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(2, 1, 2): 3, (2, 2, 1): -3, (3, 1, 1): 1}, {(1, 1, 3): -1, (1, 2, 2): 4, (2, 1, 2): -3, (1, 3, 1): -1, (2, 2, 1): 1}]
primitive elements of z-value-shuffle:
[{(3, 1, 1): 1, (1, 3, 1): -2, (1, 1, 3): 1}, {(2, 2, 1): 1, (2, 1, 2): -2, (1, 2, 2): 1}]


In [5]:
compare(1,4)

primitive elements of xy-shuffle:
[{(1, 1, 1, 1, 0): 1, (1, 1, 1, 0, 1): -4, (1, 1, 0, 1, 1): 6, (1, 0, 1, 1, 1): -4, (0, 1, 1, 1, 1): 1}]
pi_y( primitive elements of xy-shuffle ):
[{(1, 1, 1, 2): -4, (1, 1, 2, 1): 6, (1, 2, 1, 1): -4, (2, 1, 1, 1): 1}]
primitive elements of z-value-shuffle:
[{(2, 1, 1, 1): -1, (1, 2, 1, 1): 3, (1, 1, 2, 1): -3, (1, 1, 1, 2): 1}]
